In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("fact_order_lines.csv")
data2 = pd.read_csv("dim_targets_orders.csv")

In [3]:
data.head()

,order_id,order_placement_date,customer_id,product_id,order_qty,agreed_delivery_date,actual_delivery_date,delivery_qty,In Full,On Time,On Time In Full
0,FMR32603203,01-Mar-22,789603,25891203,460,02-Mar-22,03-Mar-22,460,1,0,1
1,FMR33401603,01-Mar-22,789401,25891203,429,03-Mar-22,03-Mar-22,429,1,1,1
2,FMR32320302,01-Mar-22,789320,25891203,347,02-Mar-22,02-Mar-22,347,1,1,1
3,FMR33320501,01-Mar-22,789320,25891203,187,03-Mar-22,03-Mar-22,150,0,1,0
4,FMR34220601,01-Mar-22,789220,25891203,235,04-Mar-22,04-Mar-22,235,1,1,1


In [4]:
data['order_id'].nunique()

31729

In [5]:
data.order_id.value_counts()


FMR323221602     9
FAUG812220603    9
FAUG830903603    8
FAUG815503603    8
FJUL72221603     8
                ..
FMY518220202     1
FMY518503202     1
FMY517401202     1
FMY517703202     1
FAUG831202303    1
Name: order_id, Length: 31729, dtype: int64

In [ ]:
df = data.drop(columns=["order_id","product_id","order_qty","agreed_delivery_date","actual_delivery_date","delivery_qty"], axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['order_placement_date'] = pd.to_datetime(df['order_placement_date'])

In [ ]:
df['789101'].groupby('order_placement_date')['In Full'].aggregate(np.average)

In [ ]:
a = list(df["customer_id"].unique())
a.sort()
a

In [ ]:
df2 = df.copy()

In [ ]:
x = df[df["customer_id"]==a[0]]

y = x.groupby("order_placement_date")[["In Full","On Time","On Time In Full"]].aggregate(np.average)
y.index

In [ ]:

results = pd.DataFrame({
        'customer_id':[],
        "order_placement_date":[],
        "InFull":[],
        "OnTime":[],
        "OnTimeInFull":[],
        "TargetInFull":[],
        "TargetOnTime":[],
        "TargetOnTimeInFull":[]
        })

In [ ]:
df.nunique()

In [ ]:
pd.Series(data2[data2['customer_id'] == a[0]]['ontime_target%'].values)

In [ ]:
def transformData(df, a, results):
    for i in a:
        output = pd.DataFrame({
        'customer_id':[],
        "order_placement_date":[],
        "InFull":[],
        "OnTime":[],
        "OnTimeInFull":[],
        "TargetInFull":[],
        "TargetOnTime":[],
        "TargetOnTimeInFull":[]
        })
        x = df[df["customer_id"]==i]
        y = x.groupby("order_placement_date")[["In Full","On Time","On Time In Full"]].aggregate(np.average)
        output["order_placement_date"] = pd.Series(y.index)
        output['InFull'] = pd.Series(y['In Full'].values)*100
        output['OnTime'] = pd.Series(y['On Time'].values)*100
        output['OnTimeInFull'] = pd.Series(y['On Time In Full'].values)*100
        output['TargetInFull'] = data2[data2['customer_id'] == i ]['ontime_target%'].values[0]
        output['TargetOnTime'] = data2[data2['customer_id'] == i ]['infull_target%'].values[0]
        output['TargetOnTimeInFull'] = data2[data2['customer_id'] == i ]['otif_target%'].values[0]
        output["customer_id"] = i
        print(output.shape)
        print(results.shape)
        results = pd.concat([results,output],ignore_index=True)
    # results.sort_values(["customer_id","order_placement_date"],axis=1,inplace=True)
    return results


In [ ]:

kpi = transformData(df=df,a=a,results=results)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
kpi.head(200)

In [ ]:
kpi.value_counts()

In [ ]:
kpi.to_csv("C:\\Users\\lappy\\Desktop\\ali\\Power BI\\Supply Chain Challenge\\kpi.csv",index=False)